In [1]:
from pprint import pprint as pp
import re
import pandas as pd
from bolt_app import models
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from bolt_app import db

In [2]:
engine = create_engine("sqlite://")
session = Session(engine)
db.Model.metadata.create_all(engine)
session

# The most relevant attributes for us are:
  - Category
  - Thread Size
  - Material
  - Finish

In [3]:
def read_name_and_df(filename):
    # The dtype and na_filter arguments were found to help due to pandas treating the
    # string 'None' as a proxy for Nan (in pandas dtype), or None (in python types).
    seller_inventory_df = pd.read_csv(filename, dtype=str, na_filter=False)
    # This regex captures the part between 'seller-' and the next '-' as a group.
    seller_name_match = re.search(r'seller-(\w+)-', filename)
    seller_name = seller_name_match.group(1) if seller_name_match else 'Default-sellerName'
    return seller_name, seller_inventory_df

sellerA_inventory_filename = 'sample_data/seller-a-20240625-csv-updated.csv'
sellerA_name, sellerA_inventory_df = read_name_and_df(sellerA_inventory_filename)

print(f'Seller name is {sellerA_name}, the raw data looks like this:')
sellerA_inventory_df.head()

Seller name is a, the raw data looks like this:


,product_id,description,thread_size,material,finish,quantity,price,category
0,A001,M10-1.5 X 100 HCS DIN 931 8.8 PLN,M10-1.5,Steel,Plain,500,0.75,Hex Cap Screw
1,A002,1/4-20 X 2 GR5 TEF BLU HEX HEAD CAP SC,1/4-20,Steel,Teflon Blue,1000,0.50,Hex Cap Screw
2,A003,M12-1.75 X 220 HCS DIN 931 10.9 PLN,M12-1.75,Steel,Plain,750,1.25,Hex Cap Screw
3,A004,M14-2 X 80 HCS DIN 931 8.8 ZC,M14-2,Steel,Zinc,600,1.10,Hex Cap Screw
4,A005,M8-1.25 X 60 HCS DIN 931 8.8 ZC,M8-1.25,Steel,Zinc,1200,0.40,Hex Cap Screw


In [4]:
sellerA_inventory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   5 non-null      object
 1   description  5 non-null      object
 2   thread_size  5 non-null      object
 3   material     5 non-null      object
 4   finish       5 non-null      object
 5   quantity     5 non-null      object
 6   price        5 non-null      object
 7   category     5 non-null      object
dtypes: object(8)
memory usage: 452.0+ bytes


Let us add this seller to our database:

In [5]:
sellerA = models.Seller(
    name=sellerA_name
)
session.add(sellerA)
session.commit()
sellerA.to_dict()

{'id': 1, 'name': 'a'}

This Seller "A" has the following attributes which match match:
  - category
  - thread_size
  - material
  - finish

We can map those directly to our objects fields

In [6]:
def df_to_models_A(row, seller):
    row_dict = row.to_dict()
    
    fastener = models.Fastener(**{
        'category': row_dict.get('category', 'Default-category'),
        'thread_size': row_dict.get('thread_size', 'Default-thread_size'),
        'material': row_dict.get('material', 'Default-material'),
        'finish': row_dict.get('finish', 'Default-finish'),
        'seller': seller
    })
    return fastener

sellerA_inventory_models = sellerA_inventory_df.apply(df_to_models_A, axis=1, seller=sellerA).to_list()
sellerA_inventory_models

[<Fastener (transient 4662886128)>,
 <Fastener (transient 4663577920)>,
 <Fastener (transient 4663579696)>,
 <Fastener (transient 4663579888)>,
 <Fastener (transient 4663579936)>]

We can then commit these to the session and have them persisted to the db:

In [7]:
session.add_all(sellerA_inventory_models)
session.commit()

And then we can select to ensure they are in the db:

In [8]:
stmt = db.select(models.Fastener)
for fastener in session.scalars(stmt):
    print(fastener.to_dict())

{'id': 1, 'category': 'Hex Cap Screw', 'thread_size': 'M10-1.5', 'material': 'Steel', 'finish': 'Plain', 'seller': {'id': 1, 'name': 'a'}}
{'id': 2, 'category': 'Hex Cap Screw', 'thread_size': '1/4-20', 'material': 'Steel', 'finish': 'Teflon Blue', 'seller': {'id': 1, 'name': 'a'}}
{'id': 3, 'category': 'Hex Cap Screw', 'thread_size': 'M12-1.75', 'material': 'Steel', 'finish': 'Plain', 'seller': {'id': 1, 'name': 'a'}}
{'id': 4, 'category': 'Hex Cap Screw', 'thread_size': 'M14-2', 'material': 'Steel', 'finish': 'Zinc', 'seller': {'id': 1, 'name': 'a'}}
{'id': 5, 'category': 'Hex Cap Screw', 'thread_size': 'M8-1.25', 'material': 'Steel', 'finish': 'Zinc', 'seller': {'id': 1, 'name': 'a'}}


Now let us turn to the other seller sample data:

In [9]:
sellerB_inventory_filename = 'sample_data/seller-b-20240625-csv-updated.csv'
sellerB_name, sellerB_inventory_df = read_name_and_df(sellerB_inventory_filename)

print(f'Seller name is {sellerB_name}, the raw data looks like this:')
sellerB_inventory_df.head()

Seller name is b, the raw data looks like this:


,item_number,product_name,threading,composition,surface_treatment,stock,unit_cost,product_category
0,B001,M12-1.75 X 220 HCS DIN 931 10.9 PLN,M12-1.75,Steel,Plain,700,1.30,Hex Cap Screw
1,B002,1-8 X 1 3/4 HEX CAP SCREW GR 5 ZC,1-8,Steel,Zinc,250,2.00,Hex Cap Screw
2,B003,1/4-20 X 2 1/4 HEX CAP SCREW GR 2 STL PLN MANU...,1/4-20,Steel,Plain,1500,0.35,Hex Cap Screw
3,B004,1/4-20 X 1/2 HEX CAP SCREW ALUM,1/4-20,Aluminum,None,2000,0.30,Hex Cap Screw
4,B005,1/4-20 X 5-1/4 HEX CAP SCREW GR 5 PLN,1/4-20,Steel,Plain,600,0.70,Hex Cap Screw


Let us add this new seller to the db:

In [10]:
sellerB = models.Seller(
    name=sellerB_name
)
session.add(sellerB)
session.commit()
sellerB.to_dict()

{'id': 2, 'name': 'b'}

Since the field names are not exact we can find the best match:
  - product_category -> category
  - threading -> thread_size
  - composition -> material
  - surface_treatment -> finish

In [11]:
def df_to_models_B(row, seller):
    row_dict = row.to_dict()
    
    fastener = models.Fastener(**{
        'category': row_dict.get('product_category', 'Default-category'),
        'thread_size': row_dict.get('threading', 'Default-thread_size'),
        'material': row_dict.get('composition', 'Default-material'),
        'finish': row_dict.get('surface_treatment', 'Default-finish'),
        'seller': seller
    })
    return fastener

sellerB_inventory_models = sellerB_inventory_df.apply(df_to_models_B, axis=1, seller=sellerB).to_list()
sellerB_inventory_models

[<Fastener (transient 4663705872)>,
 <Fastener (transient 4663705776)>,
 <Fastener (transient 4663706208)>,
 <Fastener (transient 4663705920)>,
 <Fastener (transient 4663706256)>]

In [12]:
session.add_all(sellerB_inventory_models)
session.commit()

In [13]:
stmt = db.select(models.Fastener)
for fastener in session.scalars(stmt):
    print(fastener.to_dict())

{'id': 1, 'category': 'Hex Cap Screw', 'thread_size': 'M10-1.5', 'material': 'Steel', 'finish': 'Plain', 'seller': {'id': 1, 'name': 'a'}}
{'id': 2, 'category': 'Hex Cap Screw', 'thread_size': '1/4-20', 'material': 'Steel', 'finish': 'Teflon Blue', 'seller': {'id': 1, 'name': 'a'}}
{'id': 3, 'category': 'Hex Cap Screw', 'thread_size': 'M12-1.75', 'material': 'Steel', 'finish': 'Plain', 'seller': {'id': 1, 'name': 'a'}}
{'id': 4, 'category': 'Hex Cap Screw', 'thread_size': 'M14-2', 'material': 'Steel', 'finish': 'Zinc', 'seller': {'id': 1, 'name': 'a'}}
{'id': 5, 'category': 'Hex Cap Screw', 'thread_size': 'M8-1.25', 'material': 'Steel', 'finish': 'Zinc', 'seller': {'id': 1, 'name': 'a'}}
{'id': 6, 'category': 'Hex Cap Screw', 'thread_size': 'M12-1.75', 'material': 'Steel', 'finish': 'Plain', 'seller': {'id': 2, 'name': 'b'}}
{'id': 7, 'category': 'Hex Cap Screw', 'thread_size': '1-8', 'material': 'Steel', 'finish': 'Zinc', 'seller': {'id': 2, 'name': 'b'}}
{'id': 8, 'category': 'Hex Ca

The above analysis was made after crafting the models in the app, and was then used to guide the development of the ingestion modules.